In [1]:
%load_ext autoreload
%autoreload 2
%config InlineBackend.figure_format = 'retina'

In [2]:
import scanpy as sc 
import sys 
import umap
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from tqdm import tqdm
import glob

sys.path.append('../../src')

from spaceoracle import SpaceTravLR

from spaceoracle.tools.network import RegulatoryFactory
from spaceoracle.gene_factory import GeneFactory

In [3]:
base_dir = '/Users/koush/Desktop/'

co_grn = RegulatoryFactory(
    colinks_path=base_dir + 'training_data/slideseq_mouse_lymphnode_colinks.pkl',
    annot='cell_type_int'
)

adata = sc.read_h5ad('/Users/koush/Desktop/training_data/slideseq_mouse_lymphnode.h5ad')
adata

AnnData object with n_obs × n_vars = 22227 × 3128
    obs: 'cell_type', 'cell_type_int', 'celltype_label_tangram'
    uns: 'cell_thresholds', 'cell_type_colors', 'cell_type_int_colors', 'received_ligands', 'received_ligands_tfl'
    obsm: 'X_spatial', 'spatial', 'spatial_unscaled', 'tangram_ct_pred'
    layers: 'imputed_count', 'normalized_count', 'raw_count'

In [4]:
gf = GeneFactory.from_json(
    adata, 
    '/Volumes/SSD/lasso_runs/slideseq_mouse_lymphnode/run_params.json', 
    override_params={'save_dir': '/Volumes/SSD/lasso_runs/slideseq_mouse_lymphnode'}
)

In [5]:
gf.load_betas(subsample=None, float16=True)

In [39]:
ko_Tgfb1_Foxp3 = gf.perturb(
    target=['Tgfb1', 'Foxp3'],
    n_propagation=4,
    gene_expr=[0, 0],
)

Tgfb1_4n_0x_Foxp3_4n_0x


In [40]:
ko_Tgfb1 = gf.perturb(
    target=['Tgfb1'],
    n_propagation=4,
    gene_expr=[0],
)

Tgfb1_4n_0x


In [41]:
ko_Foxp3 = gf.perturb(
    target=['Foxp3'],
    n_propagation=4,
    gene_expr=[0],
)

Foxp3_4n_0x


In [43]:
ko_Tgfb1[['Tgfb1', 'Foxp3']].sum()

Tgfb1     0.000000
Foxp3    90.220879
dtype: float64

In [48]:
(ko_Tgfb1_Foxp3 - adata.to_df(layer='imputed_count'))[['Bcl6']].mean()

Bcl6    0.000026
dtype: float64

In [49]:
(ko_Tgfb1 - adata.to_df(layer='imputed_count'))[['Bcl6']].mean()

Bcl6    6.255854e-11
dtype: float64

In [51]:
(ko_Foxp3 - adata.to_df(layer='imputed_count'))[['Bcl6']].mean()

Bcl6    0.000026
dtype: float64

In [54]:
(ko_Tgfb1_Foxp3 - ko_Tgfb1).mean().sort_values(ascending=False, key=abs).head(10)

Lyz2     0.009353
Apoe     0.005224
Fth1     0.005043
Foxp3   -0.004059
Cd74    -0.001687
Il2ra    0.001360
Ccl22    0.000841
Tpr     -0.000625
Clu      0.000611
Banf1    0.000604
dtype: float64

In [24]:
splashed = gf.splash_betas('Tgfb1')

In [25]:
splashed.join(adata.obs.cell_type).query('cell_type=="Th2"').drop(
    columns=['cell_type']).mean().sort_values(ascending=False, key=abs).head(10)

beta_C3        0.0
beta_Pax5      0.0
beta_Klf6      0.0
beta_Lgals9    0.0
beta_Maff      0.0
beta_Mdk       0.0
beta_Mef2c     0.0
beta_Mif       0.0
beta_Nampt     0.0
beta_Ncl       0.0
dtype: float64

In [27]:
splashed.join(adata.obs.cell_type).query('cell_type=="Treg"').drop(
    columns=['cell_type']).mean().sort_values(ascending=False, key=abs).head(10)

beta_Tbp     -0.248924
beta_Klf6     0.238214
beta_Rest    -0.133313
beta_Taf1    -0.091260
beta_Maff    -0.078346
beta_Prdm1    0.059880
beta_Pax5     0.056383
beta_Hes7     0.045915
beta_Usf1    -0.043392
beta_Mef2c    0.034786
dtype: float64

In [11]:
beta_ligands = [i for i in splashed.columns if i.replace('beta_', '') in gf.ligands]
splashed.join(adata.obs.cell_type).query('cell_type=="Th2"').drop(
    columns=['cell_type']).mean().loc[beta_ligands].sort_values(ascending=False, key=abs).head(20)

Series([], dtype: float64)

In [9]:
splashed.join(adata.obs.cell_type).query('cell_type=="Tfh"').drop(
    columns=['cell_type']).mean().loc[beta_ligands].sort_values(ascending=False, key=abs).head(20)

beta_Mif       -4.921630e-04
beta_Lgals9    -2.080178e-04
beta_Mdk        4.491973e-05
beta_C3         1.183881e-05
beta_Il16       3.606420e-06
beta_Ptn       -3.537062e-07
beta_Il2        2.328585e-07
beta_Ptprc     -2.172864e-07
beta_Il15       1.806859e-07
beta_Cd22      -1.241398e-07
beta_Nampt      8.697442e-08
beta_Ccl22      5.201300e-08
beta_Ccl5       3.407485e-08
beta_C4b       -3.300440e-08
beta_Gzma       2.060369e-08
beta_Tgfb1      1.282645e-08
beta_Angptl4   -3.757200e-09
beta_Ccl17      3.001512e-09
beta_Itgb1      1.319908e-09
beta_Ccl8      -6.487811e-11
dtype: float64

GATA3_0n_0x_IL4_0n_2x


OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.
